In [7]:
# Imports
import gym
import numpy as np

In [8]:
# Task 0 Monte Carlo
def run_episode(env, max_steps, policy):
    """
    Runs an episode of the environment
    Args:
        env: the openAI environment instance
        max_steps: maximum number of steps per episode

    Return:
        episode_results: np.ndarray of integers shape (state, reward)
    """
    state = env.reset()
    episode_results = []

    # Run each episode until we reach max_steps
    for step in range(max_steps):
        action = policy(state)
        next_state, reward, done, _ = env.step(action)
        episode_results.append([state, reward])
        if done:
            break

        state = next_state

    return np.array(episode_results, dtype=int)


def monte_carlo(env, V, policy, episodes=5000, max_steps=100, alpha=0.1,
                gamma=0.99):
    """
    Args:
        env: the openAI environment instance
        V: np.ndarray shape (s,) containing the value estimate
        policy: a function that takes in a state and returns the next action
        episodes: total number of episodes to train over
        max_steps: maximum number of steps per episode
        alpha: learning rate
        gamma: discount rate

    Returns:
        V: the updated value estimate
    """
    # Loop through our episodes
    for episode in range(episodes):
        cumulative_reward = 0
        episode_results = run_episode(env, max_steps, policy)
        # Perform Monte Carlo Algorithm from finish to start
        for time in reversed(range(0, len(episode_results))):
            state, reward = episode_results[time]
            cumulative_reward = gamma * cumulative_reward + reward
            if state not in episode_results[:episode, 0]:
                V[state] = V[state] + alpha * (cumulative_reward - V[state])

    return V

In [9]:
# 0-main
np.random.seed(0)

env = gym.make('FrozenLake8x8-v0')
LEFT, DOWN, RIGHT, UP = 0, 1, 2, 3

def policy(s):
    p = np.random.uniform()
    if p > 0.5:
        if s % 8 != 7 and env.desc[s // 8, s % 8 + 1] != b'H':
            return RIGHT
        elif s // 8 != 7 and env.desc[s // 8 + 1, s % 8] != b'H':
            return DOWN
        elif s // 8 != 0 and env.desc[s // 8 - 1, s % 8] != b'H':
            return UP
        else:
            return LEFT
    else:
        if s // 8 != 7 and env.desc[s // 8 + 1, s % 8] != b'H':
            return DOWN
        elif s % 8 != 7 and env.desc[s // 8, s % 8 + 1] != b'H':
            return RIGHT
        elif s % 8 != 0 and env.desc[s // 8, s % 8 - 1] != b'H':
            return LEFT
        else:
            return UP

V = np.where(env.desc == b'H', -1, 1).reshape(64).astype('float64')
np.set_printoptions(precision=4)
env.seed(0)
print(monte_carlo(env, V, policy).reshape((8, 8)))

[[ 0.81    0.9     0.4783  0.4305  0.3874  0.4305  0.6561  0.9   ]
 [ 0.9     0.729   0.5905  0.4783  0.5905  0.2824  0.2824  0.3874]
 [ 1.      0.5314  0.729  -1.      1.      0.3874  0.2824  0.4305]
 [ 1.      0.5905  0.81    0.9     1.     -1.      0.3874  0.6561]
 [ 1.      0.6561  0.81   -1.      1.      1.      0.729   0.5314]
 [ 1.     -1.     -1.      1.      1.      1.     -1.      0.9   ]
 [ 1.     -1.      1.      1.     -1.      1.     -1.      1.    ]
 [ 1.      1.      1.     -1.      1.      1.      1.      1.    ]]


In [10]:
# Task 1 - TD(λ)
def td_lambtha(env, V, policy, lambtha, episodes=5000, max_steps=100,
               alpha=0.1, gamma=0.99):
    """
    Args:
        env: the openAI environment instance
        V: numpy.ndarray of shape (s,) containing the value estimate
        policy: function that takes in state and returns  next action to take
        lambtha: eligibility trace factor
        episodes: total number of episodes to train over
        max_steps: maximum number of steps per episode
        alpha: learning rate
        gamma: discount rate
    Returns:
        V: the updated value estimate
    """
    eligibility_trace = np.zeros_like(V)
    for episode in range(episodes):
        state = env.reset()

        for step in range(max_steps):
            action = policy(state)
            next_state, reward, done, _ = env.step(action)

            # TD error
            delta = reward + (gamma * V[next_state] - V[state])

            # Update eligibility trace
            eligibility_trace *= (gamma * lambtha)
            eligibility_trace[state] += 1

            # Update value estimate
            V += delta * alpha * eligibility_trace

            state = next_state

            if done or step > max_steps:
                break

    return V

In [11]:
# 1-main
np.random.seed(0)

env = gym.make('FrozenLake8x8-v0')
LEFT, DOWN, RIGHT, UP = 0, 1, 2, 3

def policy(s):
    p = np.random.uniform()
    if p > 0.5:
        if s % 8 != 7 and env.desc[s // 8, s % 8 + 1] != b'H':
            return RIGHT
        elif s // 8 != 7 and env.desc[s // 8 + 1, s % 8] != b'H':
            return DOWN
        elif s // 8 != 0 and env.desc[s // 8 - 1, s % 8] != b'H':
            return UP
        else:
            return LEFT
    else:
        if s // 8 != 7 and env.desc[s // 8 + 1, s % 8] != b'H':
            return DOWN
        elif s % 8 != 7 and env.desc[s // 8, s % 8 + 1] != b'H':
            return RIGHT
        elif s % 8 != 0 and env.desc[s // 8, s % 8 - 1] != b'H':
            return LEFT
        else:
            return UP

V = np.where(env.desc == b'H', -1, 1).reshape(64).astype('float64')
np.set_printoptions(precision=4)
print(td_lambtha(env, V, policy, 0.9).reshape((8, 8)))


[[-0.8455 -0.8293 -0.8007 -0.7884 -0.7922 -0.7395 -0.7184 -0.7291]
 [-0.8697 -0.8663 -0.8324 -0.8601 -0.8517 -0.7871 -0.7165 -0.707 ]
 [-0.9069 -0.9047 -0.921  -1.     -0.8818 -0.87   -0.7787 -0.7885]
 [-0.9262 -0.9284 -0.9416 -0.9952 -0.984  -1.     -0.9025 -0.8248]
 [-0.9205 -0.9471 -0.9595 -1.     -0.9654 -0.9457 -0.9259 -0.6976]
 [-0.9515 -1.     -1.      0.6371 -0.9658 -0.9013 -1.     -0.1736]
 [-0.962  -1.     -0.3394  0.2564 -1.     -0.6066 -1.      0.292 ]
 [-0.9403 -0.9457 -0.8711 -1.      1.      0.6161  0.9151  1.    ]]


In [12]:
# Task 2 Sarsa(λ)
def sarsa_lambtha(env, Q, lambtha, episodes=5000, max_steps=100, alpha=0.1,
                  gamma=0.99, epsilon=1, min_epsilon=0.1, epsilon_decay=0.05):
    """
    Args:
        env: the openAI environment instance
        Q: numpy.ndarray of shape (s,a) containing the Q table
        lambtha: eligibility trace factor
        episodes: total number of episodes to train over
        max_steps: maximum number of steps per episode
        alpha: learning rate
        gamma: discount rate
        epsilon: initial threshold for epsilon greedy
        min_epsilon: minimum value that epsilon should decay to
        epsilon_decay: decay rate for updating epsilon between episodes

    Returns:
        Q: the updated Q table
    """
    eligibility_trace = np.zeros_like(Q)
    for episode in range(episodes):
        state = env.reset()
        episode_done = False

        action = epsilon_greedy(Q, state, epsilon)

        epsilon = max(min_epsilon, epsilon - epsilon_decay)

        for step in range(max_steps):
            n_s, reward, episode_done, _ = env.step(action)
            n_a = epsilon_greedy(Q, n_s, epsilon)

            # Calculate TD error
            td_error = reward + gamma * Q[n_s][n_a] - Q[state][action]

            # Update eligibility trace
            eligibility_trace *= lambtha * gamma
            eligibility_trace[state][action] = 1.0

            # Update Q values
            Q += alpha * td_error * eligibility_trace

            state = n_s
            action = n_a

            if episode_done:
                break

    return Q


def epsilon_greedy(Q, state, epsilon):
    if np.random.uniform(0, 1) < epsilon:
        return np.random.choice(len(Q[state]))
    else:
        return np.argmax(Q[state])

In [13]:
#2-main
np.random.seed(0)
env = gym.make('FrozenLake8x8-v0')
Q = np.random.uniform(size=(64, 4))
np.set_printoptions(precision=4)
print(sarsa_lambtha(env, Q, 0.9))

[[0.5929 0.6669 0.5685 0.5609]
 [0.5401 0.5712 0.6599 0.5251]
 [0.6508 0.5524 0.528  0.5659]
 [0.4552 0.5299 0.4191 0.4677]
 [0.4823 0.5262 0.64   0.5597]
 [0.5016 0.6507 0.493  0.4886]
 [0.3519 0.6356 0.3198 0.5394]
 [0.6511 0.5191 0.3894 0.5458]
 [0.5743 0.7175 0.6175 0.6261]
 [0.5971 0.6942 0.6236 0.6137]
 [0.6725 0.4886 0.5165 0.5232]
 [0.3607 0.3682 0.4257 0.364 ]
 [0.5097 0.4253 0.6443 0.4497]
 [0.6604 0.465  0.4425 0.4943]
 [0.6643 0.2929 0.5099 0.2444]
 [0.2676 0.2762 0.6682 0.1751]
 [0.6628 0.7516 0.6277 0.6218]
 [0.7255 0.6231 0.5879 0.5897]
 [0.7109 0.5437 0.5272 0.57  ]
 [0.2828 0.1202 0.2961 0.1187]
 [0.4761 0.6666 0.3854 0.4799]
 [0.6227 0.4961 0.6864 0.3488]
 [0.6043 0.7512 0.5106 0.6096]
 [0.2377 0.7803 0.3768 0.2804]
 [0.689  0.8001 0.6844 0.626 ]
 [0.7097 0.77   0.7208 0.6442]
 [0.6708 0.8351 0.6862 0.6964]
 [0.6973 0.8575 0.6333 0.5406]
 [0.7386 0.8886 0.7274 0.6494]
 [0.8811 0.5813 0.8817 0.6925]
 [0.7839 0.5543 0.6725 0.674 ]
 [0.5296 0.7939 0.1599 0.4759]
 [0.6798